# Nguyễn Hoàng Tân - 21521413
# CS410 - Lab06

In [1]:
!git clone https://github.com/Dev-Aligator/UIT

Cloning into 'UIT'...
remote: Enumerating objects: 21993, done.
remote: Counting objects: 100% (1648/1648), done.
remote: Compressing objects: 100% (976/976), done.
remote: Total 21993 (delta 520), reused 1625 (delta 498), pack-reused 20345
Receiving objects: 100% (21993/21993), 229.70 MiB | 31.54 MiB/s, done.
Resolving deltas: 100% (533/533), done.


In [2]:
import sys
sys.path.append('/kaggle/working/UIT/CS410.O21.KHCL/Assignments/BT6_TransformerModel/src') 
import torch
import torch.nn as nn
from model import Transformer
from config import get_config, get_weights_file_path
from train import get_model, get_ds, greedy_decode
import altair as alt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

2024-06-17 15:00:30.193285: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 15:00:30.193387: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 15:00:30.327139: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [4]:
config = get_config()
train_dataloader, val_dataloader, vocab_src, vocab_tgt = get_ds(config)
model = get_model(config, vocab_src.get_vocab_size(), vocab_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = get_weights_file_path(config, f"10")
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

Generating train split:   0%|          | 0/203272 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25409 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/25409 [00:00<?, ? examples/s]

Max length of source sentence: 53
Max length of target sentence: 64


<All keys matched successfully>

In [5]:
def load_next_batch():
    # Load a sample batch from the validation set
    batch = next(iter(val_dataloader))
    encoder_input = batch["encoder_input"].to(device)
    encoder_mask = batch["encoder_mask"].to(device)
    decoder_input = batch["decoder_input"].to(device)
    decoder_mask = batch["decoder_mask"].to(device)

    encoder_input_tokens = [vocab_src.id_to_token(idx) for idx in encoder_input[0].cpu().numpy()]
    decoder_input_tokens = [vocab_tgt.id_to_token(idx) for idx in decoder_input[0].cpu().numpy()]

    # check that the batch size is 1
    assert encoder_input.size(
        0) == 1, "Batch size must be 1 for validation"

    model_out = greedy_decode(
        model, encoder_input, encoder_mask, vocab_src, vocab_tgt, config['seq_len'], device)
    
    return batch, encoder_input_tokens, decoder_input_tokens

In [6]:
def mtx2df(m, max_row, max_col, row_tokens, col_tokens):
    return pd.DataFrame(
        [
            (
                r,
                c,
                float(m[r, c]),
                "%.3d %s" % (r, row_tokens[r] if len(row_tokens) > r else "<blank>"),
                "%.3d %s" % (c, col_tokens[c] if len(col_tokens) > c else "<blank>"),
            )
            for r in range(m.shape[0])
            for c in range(m.shape[1])
            if r < max_row and c < max_col
        ],
        columns=["row", "column", "value", "row_token", "col_token"],
    )

def get_attn_map(attn_type: str, layer: int, head: int):
    if attn_type == "encoder":
        attn = model.encoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "decoder":
        attn = model.decoder.layers[layer].self_attention_block.attention_scores
    elif attn_type == "encoder-decoder":
        attn = model.decoder.layers[layer].cross_attention_block.attention_scores
    return attn[0, head].data

def attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len):
    df = mtx2df(
        get_attn_map(attn_type, layer, head),
        max_sentence_len,
        max_sentence_len,
        row_tokens,
        col_tokens,
    )
    return (
        alt.Chart(data=df)
        .mark_rect()
        .encode(
            x=alt.X("col_token", axis=alt.Axis(title="")),
            y=alt.Y("row_token", axis=alt.Axis(title="")),
            color="value",
            tooltip=["row", "column", "value", "row_token", "col_token"],
        )
        #.title(f"Layer {layer} Head {head}")
        .properties(height=400, width=400, title=f"Layer {layer} Head {head}")
        .interactive()
    )

def get_all_attention_maps(attn_type: str, layers: list[int], heads: list[int], row_tokens: list, col_tokens, max_sentence_len: int):
    charts = []
    for layer in layers:
        rowCharts = []
        for head in heads:
            rowCharts.append(attn_map(attn_type, layer, head, row_tokens, col_tokens, max_sentence_len))
        charts.append(alt.hconcat(*rowCharts))
    return alt.vconcat(*charts)

In [9]:
batch, encoder_input_tokens, decoder_input_tokens = load_next_batch()
print(f'Source: {batch["src_text"][0]}')
print(f'Target: {batch["tgt_text"][0]}')
sentence_len = encoder_input_tokens.index("[PAD]")

Source: Tell Tom I'll call later this afternoon.
Target: nói với tom tôi sẽ gọi vào chiều nay.


In [10]:
layers = [0, 1, 2]
heads = [0, 1, 2, 3, 4, 5, 6, 7]

# Encoder Self-Attention
get_all_attention_maps("encoder", layers, heads, encoder_input_tokens, encoder_input_tokens, min(20, sentence_len))


alt.VConcatChart(...)

In [12]:
# Decoder Self-Attention
get_all_attention_maps("decoder", layers, heads, decoder_input_tokens, decoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)

In [11]:
# cross attention
get_all_attention_maps("encoder-decoder", layers, heads, encoder_input_tokens, decoder_input_tokens, min(20, sentence_len))

alt.VConcatChart(...)

# Summary

### Encoder Self-Attention Visualization

1. **Diagonal Dominance**:
    - High attention values along the diagonals indicate strong self-attention scores where each token has a high dot product with itself.

2. **Token-Specific Attention**:
    - Start-of-sentence (SOS) and end-of-sentence (EOS) tokens in head 0 of layer 0 show low attention to other words.
    - Other heads display varying mappings, some with minor associations.

3. **Attention Values**:
    - Attention scores highlight strong relationships, such as "especially" with itself and "now."

4. **Layer and Head Variability**:
    - Different heads and layers capture diverse aspects and relationships of tokens due to varying \( W_Q \), \( W_K \), and \( W_V \) matrices.

These visualizations confirm the encoder effectively captures both individual token importance and complex inter-token relationships.


### Decoder Self-Attention Visualization

1. **Token-Specific Attention**:
    - Similar to the encoder, high attention values are seen for tokens with themselves, ensuring fluency and consistency in generation.

2. **Multiple Heads and Layers**:
    - Multiple attention heads learn different mappings between words.
    - Different layers capture various relationships due to distinct \( W_Q \), \( W_K \), and \( W_V \) matrices in each layer.

These visualizations indicate that the decoder self-attention effectively maintains output coherence and consistency, with each head and layer capturing diverse token relationships.

### Cross-Attention Visualization

1. **Interaction Between Encoder and Decoder**:
    - Cross-attention represents the interaction between the encoder and decoder, crucial for tasks like translation.

2. **Attention Mechanism**:
    - Keys and values come from the encoder, while queries come from the decoder.
    - This mechanism enables the model to align and relate input and output sequences.

3. **Translation Task**:
    - Cross-attention is where the model learns to map the input sentence to the output sentence, effectively performing translation.

These visualizations show how cross-attention facilitates the translation process by dynamically relating the encoder's input sequence to the decoder's output sequence.